In [1]:
import copy, math
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.datasets import make_moons, make_circles, make_blobs, load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from torchsummary import summary

from qiskit import Aer, IBMQ
from qiskit.utils import QuantumInstance
from qiskit.utils import algorithm_globals
from qiskit.algorithms.optimizers import COBYLA, NELDER_MEAD, SLSQP, SPSA
from qiskit.opflow import Z, I, StateFn, PauliExpectation, Gradient, NaturalGradient
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit_machine_learning.neural_networks import SamplerQNN, EstimatorQNN, OpflowQNN, CircuitQNN, TwoLayerQNN
from qiskit_machine_learning.neural_networks import EffectiveDimension
from qiskit_machine_learning.algorithms.classifiers import VQC, NeuralNetworkClassifier
from qiskit.quantum_info import SparsePauliOp
from qiskit_machine_learning.connectors import TorchConnector

# Set seed for random generators
algorithm_globals.random_seed = 42

from IPython.display import clear_output

In [2]:
from Components.circuits import preTrainedBlockGenerator, layerwise_training, featureMapGenerator, AnsatzGenerator
from Components.data import breast_cancer, iris, fetch_mnist
from Components.utils import plot_loss, score, parity, classification_callback, plot_objfn_range, result_to_objfun_dataframes, save_results
from Components.train import create_qnn, sampling_experiment, train, train_batch

In [3]:
backend = Aer.get_backend('statevector_simulator')

q_instance = QuantumInstance(
    backend, 
    shots = 100, 
    seed_simulator = 2718, 
    seed_transpiler = 2718,
    )

In [4]:
X_train, X_val, y_train, y_val = fetch_mnist(PCA_n = 4, data_size=500)

X_train_t = torch.from_numpy(X_train).to(torch.float32)
y_train_t = torch.from_numpy(y_train).to(torch.float32)
X_val_t = torch.from_numpy(X_val).to(torch.float32)
y_val_t = torch.from_numpy(y_val).to(torch.float32)

Training set: 350 samples
Testing set: 150 samples
Number of features: 4
Classes:[0 1]; Encoded as: [-1  1]


In [5]:
train_dataset = TensorDataset(X_train_t, y_train_t)
val_dataset = TensorDataset(X_val_t, y_val_t)
train_dataloader = DataLoader(train_dataset, batch_size=1000)
val_dataloader = DataLoader(val_dataset, batch_size=1000)

In [6]:
def show_loss(losses):
    plt.figure(figsize=(12, 8))
    plt.plot(range(len(losses)), losses)
    plt.xlabel("Iteration")
    plt.ylabel("Loss")
    plt.show()

In [7]:
FEATURE_DIM = X_train.shape[-1]
max_qubit = FEATURE_DIM
MAX_REPS = 9
MIN_REPS = 1
MAX_IDENTITIES_BLOCKS = 2  # <---- Number of identity blocks, depth values of identity blocks is close to that of normal ansatz (5 qubits)
ENTANGLEMENT = 'linear'

GLOBAL_OPERATOR = Z ^ FEATURE_DIM
LOCAL_OPERATOR = (I ^ (FEATURE_DIM - 2)) ^ (Z^2) 
MAX_ITER = 120

parity = lambda x: "{:b}".format(x).count("1") % 2  # optional interpret function, can't be used for EstimatorQNN

## Method 3

In [8]:
feature_map = featureMapGenerator(FEATURE_DIM)
identity_block = preTrainedBlockGenerator(FEATURE_DIM, MAX_IDENTITIES_BLOCKS, entanglement=ENTANGLEMENT)
ansatz = identity_block['circuit']
qc_3 = feature_map.compose(ansatz)

In [9]:
losses_3 = []
accuracy_train_3 = []
accuracy_test_3 = []
weights_3 = []

for i in range(10):
    print(f'Training instance {i}')

    identity_block = preTrainedBlockGenerator(FEATURE_DIM, MAX_IDENTITIES_BLOCKS, entanglement=ENTANGLEMENT)
    initial_point = list(identity_block['params_values'].values())

    qnn = TwoLayerQNN(
        feature_map=feature_map,
        ansatz=ansatz,
        observable=GLOBAL_OPERATOR,
        # input_params=feature_map.parameters,
        # weight_params=anz.parameters,
        quantum_instance=q_instance
    )

    model = TorchConnector(qnn, initial_weights=initial_point)

    loss_function = nn.MSELoss()
    optimizer = torch.optim.NAdam(model.parameters(), lr=0.05)

    # model, losses, accuracy_train, accuracy_test, weights = train_batch(
    #     model, 
    #     MAX_ITER, 
    #     train_dataloader=train_dataloader, 
    #     val_dataloader=val_dataloader,
    #     optimizer = optimizer, 
    #     loss_function = loss_function
    #     )
    
    model, losses, accuracy_train, accuracy_test, weights = train(
        model, 
        MAX_ITER, 
        X_train_t,
        y_train_t,
        X_val_t,
        y_val_t,
        optimizer = optimizer, 
        loss_function = loss_function
        )

    losses_3.append(losses)
    accuracy_train_3.append(accuracy_train)
    accuracy_test_3.append(accuracy_test)
    weights_3.append(weights)

    clear_output(wait=True)

Training instance 9
Epoch      Loss                 Train Accuracy   Test Accuracy   
[ 0 ]      1.0358400344848633   0.52571          0.48667         
[ 10 ]     0.621150016784668    0.88             0.86667         
[ 20 ]     0.5653700232505798   0.89429          0.87333         
[ 30 ]     0.5534700155258179   0.89429          0.89333         
[ 40 ]     0.5498999953269958   0.89429          0.91333         
[ 50 ]     0.5472699999809265   0.90857          0.90667         
[ 60 ]     0.5467699766159058   0.92             0.90667         
[ 70 ]     0.5517899990081787   0.91714          0.89333         
[ 80 ]     0.5475299954414368   0.92             0.90667         
[ 90 ]     0.5467299818992615   0.92             0.9             
[ 100 ]    0.5470700263977051   0.92             0.9             
[ 110 ]    0.5498200058937073   0.92             0.90667         
[ 119 ]    0.5502099990844727   0.91714          0.89333         


In [10]:
pd.DataFrame(losses_3[i] for i in range(len(losses_3))).astype('float').to_csv(f'./Saves/LossFunction/m3.csv')
pd.DataFrame(accuracy_train_3[i] for i in range(len(accuracy_train_3))).to_csv(f'./Saves/Scores/Train/m3.csv')
pd.DataFrame(accuracy_test_3[i] for i in range(len(accuracy_test_3))).to_csv(f'./Saves/Scores/Test/m3.csv')
for wr in range(len(weights_3)):
    weight_record = pd.DataFrame(weights_3[wr]).astype('float')
    weight_record.to_csv(f'./Saves/Weights/m3/sample_{wr}.csv')